# Курсовая работа
## по дисциплине "Структуры и алгоритмы обработки данных"
## Выполнила студентка группы БВТ2103 Пьянова Анна Олеговна

### Задание

Написать программу, которая будет считывать данные из CSV файла, содержащего
информацию о продажах товаров в магазине. Данные в файле содержатся в следующем
формате:
| Номер заказа | Дата заказа | Название товара | Категория товара | Количество продаж | Цена
за единицу | Общая стоимость |

Необходимо:
1. Рассчитать общую выручку магазина.
2. Найти товар, который был продан наибольшее количество раз.
3. Найти товар, который принес наибольшую выручку.
4. Составить отчет, содержащий информацию об общей выручке магазина, количестве проданных единиц каждого товара и доле каждого товара в общей выручке.

Для решения задач необходимо использовать структуры данных, такие как массивы и хеш-таблицы, а также различные алгоритмы обработки данных, например, сортировку и поиск. Также необходимо учитывать возможные ошибки ввода-вывода и обрабатывать их в
соответствии с требованиями.

In [2]:
#при несоответствии ключей таблицы требованиям в условии
class KeysNoEqualError(Exception):

    def __init__(self, keys, message = "Keys don't match!"):
        self.keys = keys
        self.message = message
        super().__init__(self.message)


#при ошибочных значениях у поступивших на вход данных
class ValueInputError(Exception):
    
    def __init__(self, message = "Values don't match the format!"):
        self.message = message
        super().__init__(self.message)

In [3]:
check_keys = [
        "Номер заказа", 
        "Дата заказа",
        "Название товара",
        "Категория товара",
        "Количество продаж",
        "Цена за единицу",
        "Общая стоимость"
    ]

def check_values(item):
    #проверка на положительные числовые значения
    if (int(item["Номер заказа"]) < 0 and int(item["Количество продаж"]) < 0 
        and int(item["Цена за единицу"]) < 0 and int(item["Общая стоимость"]) < 0):
        return False
    date = item["Дата заказа"]
    #проверка на верный формат даты
    if 1 > int(date[:2]) > 31 and 1 > int(date[3:5]) > 12 and 1990 > int(date[6:]) > 2023:
        return False
    return True


In [37]:
import csv

class HashMap(object):
    #инициализация
    def __init__(self, size):
        self.data = [[]] * (size)
    
    #хэш-функция
    def hash(self, key):
        hash_key = 0
        for i in range(len(key)):
            hash_key = (hash_key + ord(key[i]) * i) % len(self.data)
        return hash_key
 
    #добавление в HashMap
    def set(self, key, value):
        index = self.hash(key)
        self.data[index].append([key, value])
        return self.data
 
    #получение значения по ключу
    def get(self, key):
        res = []
        index = self.hash(key)
        currentBucket = self.data[index]
        if currentBucket:
            for i in range(len(currentBucket)):
                if currentBucket[i][0] == key:
                    res.append(currentBucket[i][1])
        return res

    #для вывода на экран
    def __str__(self):
        return "".join(str(item) for item in self.data)

#получение данных из csv-файла
def open_csv(file_path):   
    #преобразование элементов таблицы в массив словарей
    with open(file_path, encoding='utf-8') as r_file:
        file_reader = csv.DictReader(r_file, delimiter = ";")
        items = [] #массив словарей
        for row in file_reader:
            if not check_values(row):
                raise ValueInputError
            items.append(row)
        map = HashMap(len(items)) 
        for row in items: #преобразование словарей в HashMap
            for key in row:
                map.set(key, row[key])

    #получение списка ключей (названий столбцов) таблицы
    with open(file_path, 'r', encoding='utf-8', newline='') as csvfile:
        file_reader = csv.reader(csvfile, delimiter=';')
        for row in file_reader:
            keys = row
            break

    #проверка ключей
    if keys != check_keys:
        raise KeysNoEqualError(keys)
    
    return items, map #возвращение словарей и HashMap

items, market = open_csv("table.csv")

### Задание 1

In [38]:
sum = 0

for item in market.get("Общая стоимость"):
    sum += int(item)

print("Общая выручка магазина составила", sum, ".")

Общая выручка магазина составила 241969500 .


### Задание 2

In [29]:
#реализация Дека
class Deque:

    #инициализация
    def __init__(self):
        self.items = []

    #проверка на пустоту
    def isEmpty(self):
        return self.items == []

    #добавление нового элемента в конец
    def add(self, item):
        self.items.append(item)

    #добавление нового элемента в начало
    def addLeft(self, item):
        self.items.insert(0,item)

    #извлечение элемента из конца
    def pop(self):
        return self.items.pop()

    #извлечение элемента из начала
    def popLeft(self):
        return self.items.pop(0)
    
    #возвращение последнего элемента
    def get(self):
        return self.items[len(self.items) - 1]

    #возвращение количества элементов
    def lenght(self):
        return len(self.items)

In [30]:
#сортировка по убыванию с использованием двух Деков
def sort_with_deque(array, key):
    noSort = Deque()
    sort = Deque()
    res = []

    for i in array:
        noSort.add(i)

    while not noSort.isEmpty():
        cur = noSort.pop()
        while not sort.isEmpty() and int(sort.get()[key]) > int(cur[key]):
            noSort.addLeft(sort.pop())
        sort.add(cur) #sort отсортирован по возрастанию

    while not sort.isEmpty():
        res.append(sort.pop()) #res - по убыванию

    return res

In [31]:
max_sales = sort_with_deque(items, "Количество продаж")
mx = int(max_sales[0]["Количество продаж"]) #первый элемент в отсортированном массиве

#для вывода всех товаров с наибольшим одинаковым количством продаж
for cur in max_sales:
    if int(cur["Количество продаж"]) == mx:
        print(cur["Название товара"], "был продан", cur["Количество продаж"], "раз, что является наибольшим.")

Apple iPhone 14 был продан 500 раз, что является наибольшим.
Philips Hue Smart Bulbs был продан 500 раз, что является наибольшим.


### Задание 3

In [32]:
#быстрая сортировка по убыванию
def quick_sort(array, key):
    if len(array) <= 1:
        return array
    else:
        barrier = int(array[len(array) // 2][key])
        left = [i for i in array if int(i[key]) < barrier]
        middle = [i for i in array if int(i[key]) == barrier]
        right = [i for i in array if int(i[key]) > barrier]
        
    return quick_sort(right, key) + middle + quick_sort(left, key)

In [33]:
max_price = quick_sort(items, "Общая стоимость")
mxp = int(max_price[0]["Общая стоимость"]) #первый элемент в отсортированном массиве

#для вывода всех товаров с наибольшей одинаковой общей стоимостью
for cur in max_price:
    if int(cur["Общая стоимость"]) == mxp:
        print(cur["Название товара"], "принёс выручку", cur["Общая стоимость"], ", которая является наибольшей.")

Apple iPhone 14 принёс выручку 44995000 , которая является наибольшей.


### Задание 4

In [39]:
import pandas as pd

print("Общая выручка магазина составила", sum, ".")

name = []
count = []
share = []

for item in quick_sort(items, "Общая стоимость"):
    try:
        name.append(item["Название товара"])
        count.append(item["Количество продаж"])
        share.append(str(round((float(item["Общая стоимость"]) / sum * 100), 2)) + "%")

    except ZeroDivisionError:
        print("Can't divide by zero!")
        break

report = pd.DataFrame({"Название товара": name,
                       "Количество продаж": count,
                       "Доля товара в общей выручке": share})
report


Общая выручка магазина составила 241969500 .


,Название товара,Количество продаж,Доля товара в общей выручке
0,Apple iPhone 14,500,18.6%
1,Samsung Galaxy S22 Ultra,300,12.4%
2,LG OLED TV,150,11.16%
3,Canon EOS R8,75,9.3%
4,Apple MacBook Pro 16,100,8.27%
5,Apple iPad Pro 12.9,200,7.44%
6,DJI Ronin 4D,25,5.17%
7,Canon EOS R7,50,5.17%
8,Игровые приставки,200,4.96%
9,Samsung Galaxy Book Pro,50,2.69%
